# * Import : "CDSAPPO.DIM_TIME"
    WHERE TM_KEY_YR >= 2023

## Parameter

In [1]:
import os
import glob
import configparser
import datetime as dt
import pandas as pd
import numpy as np
import xlrd
import oracledb

config = configparser.ConfigParser()
config.read('../../my_config.ini')
config.sections()

TDMDBPR_user = config['TDMDBPR']['username']
TDMDBPR_pwd = config['TDMDBPR']['password']
TDMDBPR_db = config['TDMDBPR']['db']
TDMDBPR_host = config['TDMDBPR']['host']
TDMDBPR_port = config['TDMDBPR']['port']

AKPIPRD_user = config['AKPIPRD']['username']
AKPIPRD_pwd = config['AKPIPRD']['password']
AKPIPRD_db = config['AKPIPRD']['db']
AKPIPRD_host = config['AKPIPRD']['host']
AKPIPRD_port = config['AKPIPRD']['port']

curr_dt = dt.datetime.now().date()
str_curr_dt = curr_dt.strftime('%Y%m%d')
curr_dt

datetime.date(2026, 1, 5)

### Input Parameter

In [15]:
# Input : 0 for dummy parameter

op_dir = 'data'
op_file = 'dim_time.csv'
op_file_mth = 'dim_time_mth.csv'
v_param = dict(yr_start=2023, yr_end=0)

## ETL Process...

### Step 1 : DIM_TIME
    DB -> CSV file

In [4]:
job_start_datetime = dt.datetime.now().strftime('%Y-%m-%d, %H:%M:%S')
print(f'\nJob Start... {job_start_datetime}')

with open('SQL/Import-DIM_TIME.sql', 'r') as sql_file:
    queries = sql_file.read().split(';')
    query = queries[0].strip()
    sql_file.close()

# Connect : TDMDBPR
src_dsn = f'{TDMDBPR_user}/{TDMDBPR_pwd}@{TDMDBPR_host}:{TDMDBPR_port}/{TDMDBPR_db}'
src_conn = oracledb.connect(src_dsn)
print(f'\n{TDMDBPR_db} : Connected')
src_cur = src_conn.cursor()

# Connect : AKPIPRD
tgt_dsn = f'{AKPIPRD_user}/{AKPIPRD_pwd}@{AKPIPRD_host}:{AKPIPRD_port}/{AKPIPRD_db}'
tgt_conn = oracledb.connect(tgt_dsn)
print(f'\n{AKPIPRD_db} : Connected')
tgt_cur = tgt_conn.cursor()


try:
    print(f"\nProcessing...\n\n   -> data as of (YR_Start: {v_param['yr_start']}, YR_End: {v_param['yr_end']})")

    # Create Dataframe
    src_cur.execute(query, v_param)
    rows = src_cur.fetchall()
    src_df = pd.DataFrame.from_records(rows, columns=[x[0] for x in src_cur.description])
    print(f'\n   -> src_df : {src_df.shape[0]} rows, {src_df.shape[1]} columns')
    src_cur.close()

    # Generate CSV file
    src_df.to_csv(f'{op_dir}/{op_file}', index=False, encoding='utf-8')
    print(f'\n   -> Generate "{op_file}" successfully')
    
    # Truncate
    tgt_cur.execute(f"TRUNCATE TABLE AUTOKPI.DIM_TIME")
    print(f'\n   -> TRUNCATE : "DIM_TIME" : Done !')
    
    # Insert
    tgt_cur.executemany(f"""
        INSERT INTO AUTOKPI.DIM_TIME
        (TM_KEY_DAY, DATE_ID, DATE_VALUE, DATE_DESC, DAY_DESC, DAY_SHORT, DAY_DESC_TH, DAY_SHORT_TH, DAY_NO, DAY_OF_WEEK, DAYS_IN_MONTH, TM_KEY_WK, WEEK_DESC, WEEK_YEAR, TRUE_TM_KEY_WK, TRUE_WEEK, TRUE_WEEK_DESC, TM_KEY_MTH, MONTH_ID, MONTH_SHORT, MONTH_DESC, MONTH_SHORT_TH, MONTH_DESC_TH, MONTH_NO, MONTH_BEG_DATE, MONTH_END_DATE, TM_KEY_QTR, QUARTER_DESC, QUARTER_SHORT, QUARTER_DESC_TH, QUARTER_SHORT_TH, QUARTER_NO, TM_KEY_YR, YEAR_DESC, YEAR_SHORT, YEAR_DESC_TH, YEAR_SHORT_TH, PERIODFLAG, ETL_DATE, ETL_UPDATE, WEEK_MONTH, WEEK_MONTH_DESC, TRUE_WK_BEG_DATE, TRUE_WK_END_DATE, TRUE_WK_MTH_BEG_DATE, TRUE_WK_MTH_END_DATE) 
        VALUES (:1,:2,:3,:4,:5,:6,:7,:8,:9,:10,:11,:12,:13,:14,:15,:16,:17,:18,:19,:20,:21,:22,:23,:24,:25,:26,:27,:28,:29,:30,:31,:32,:33,:34,:35,:36,:37,:38,:39,:40,:41,:42,:43,:44,:45,:46)
    """, rows)
    print(f'\n   -> INSERT : "DIM_TIME" : Done !')

    tgt_cur.close()
    tgt_conn.commit()
    

except oracledb.DatabaseError as e:
    print(f'\nError with Oracle : {e}')


finally:
    src_conn.close()
    print(f'\n{TDMDBPR_db} : Disconnected')
    tgt_conn.close()
    print(f'\n{AKPIPRD_db} : Disconnected')
    
    print(f'\nJob Done !!!')



Job Start... 2026-01-05, 13:02:18

TDMDBPR : Connected

AKPIPRD : Connected

Processing...

   -> data as of (YR_Start: 2023, YR_End: 0)

   -> src_df : 1461 rows, 46 columns

   -> Generate "dim_time.csv" successfully

   -> TRUNCATE : "DIM_TIME" : Done !

   -> INSERT : "DIM_TIME" : Done !

TDMDBPR : Disconnected

AKPIPRD : Disconnected

Job Done !!!


In [9]:
# src_df.dtypes
src_df.tail()

,TM_KEY_DAY,DATE_ID,DATE_VALUE,DATE_DESC,DAY_DESC,DAY_SHORT,DAY_DESC_TH,DAY_SHORT_TH,DAY_NO,DAY_OF_WEEK,...,YEAR_SHORT_TH,PERIODFLAG,ETL_DATE,ETL_UPDATE,WEEK_MONTH,WEEK_MONTH_DESC,TRUE_WK_BEG_DATE,TRUE_WK_END_DATE,TRUE_WK_MTH_BEG_DATE,TRUE_WK_MTH_END_DATE
1456,20261227,46383,2026-12-27,27-12-2026,Sunday,Sun,อาทิตย์,อา.,27,7,...,69,N,2025-12-22 10:06:39,2025-12-22 10:06:39,2026124,Dec 22-28,2026-12-21,2026-12-27,2026-12-21,2026-12-27
1457,20261228,46384,2026-12-28,28-12-2026,Monday,Mon,จันทร์,จ.,28,1,...,69,N,2025-12-22 10:06:39,2025-12-22 10:06:39,2026124,Dec 22-28,2026-12-28,2027-01-03,2026-12-28,2026-12-31
1458,20261229,46385,2026-12-29,29-12-2026,Tuesday,Tue,อังคาร,อ.,29,2,...,69,N,2025-12-22 10:06:39,2025-12-22 10:06:39,2026125,Dec 29-31,2026-12-28,2027-01-03,2026-12-28,2026-12-31
1459,20261230,46386,2026-12-30,30-12-2026,Wednesday,Wed,พุธ,พ.,30,3,...,69,N,2025-12-22 10:06:39,2025-12-22 10:06:39,2026125,Dec 29-31,2026-12-28,2027-01-03,2026-12-28,2026-12-31
1460,20261231,46387,2026-12-31,31-12-2026,Thursday,Thu,พฤหัสบดี,พฤ.,31,4,...,69,EMQY,2025-12-22 10:06:39,2025-12-22 10:06:39,2026125,Dec 29-31,2026-12-28,2027-01-03,2026-12-28,2026-12-31


### Step 2 : DIM_TIME_MTH
    DB -> CSV file

In [17]:
''' Monthly '''

mth_df = src_df.groupby(['TM_KEY_MTH', 'MONTH_ID', 'MONTH_SHORT', 'MONTH_DESC', 'MONTH_SHORT_TH', 'MONTH_DESC_TH', 'MONTH_NO', 'MONTH_BEG_DATE', 'MONTH_END_DATE', 'TM_KEY_QTR', 'QUARTER_DESC', 'QUARTER_SHORT', 'QUARTER_DESC_TH', 'QUARTER_SHORT_TH', 'QUARTER_NO', 'TM_KEY_YR', 'YEAR_DESC', 'YEAR_SHORT', 'YEAR_DESC_TH', 'YEAR_SHORT_TH']).agg({'ETL_DATE': 'max', 'ETL_UPDATE': 'max'}).sort_values(by=['TM_KEY_MTH']).reset_index()

# Generate CSV file
mth_df.to_csv(f'{op_dir}/{op_file_mth}', index=False, encoding='utf-8')
print(f'\n   -> Generate "{op_file_mth}" successfully')


   -> Generate "dim_time_mth.csv" successfully


In [18]:
mth_df.tail()

,TM_KEY_MTH,MONTH_ID,MONTH_SHORT,MONTH_DESC,MONTH_SHORT_TH,MONTH_DESC_TH,MONTH_NO,MONTH_BEG_DATE,MONTH_END_DATE,TM_KEY_QTR,...,QUARTER_DESC_TH,QUARTER_SHORT_TH,QUARTER_NO,TM_KEY_YR,YEAR_DESC,YEAR_SHORT,YEAR_DESC_TH,YEAR_SHORT_TH,ETL_DATE,ETL_UPDATE
43,202608,1520,Aug,August,ส.ค.,สิงหาคม,8,2026-08-01,2026-08-31,20263,...,ไตรมาส 3,ไตร 3,3,2026,2026,26,2569,69,2025-12-22 10:06:39,2025-12-22 10:06:39
44,202609,1521,Sep,September,ก.ย.,กันยายน,9,2026-09-01,2026-09-30,20263,...,ไตรมาส 3,ไตร 3,3,2026,2026,26,2569,69,2025-12-22 10:06:39,2025-12-22 10:06:39
45,202610,1522,Oct,October,ต.ค.,ตุลาคม,10,2026-10-01,2026-10-31,20264,...,ไตรมาส 4,ไตร 4,4,2026,2026,26,2569,69,2025-12-22 10:06:39,2025-12-22 10:06:39
46,202611,1523,Nov,November,พ.ย.,พฤศจิกายน,11,2026-11-01,2026-11-30,20264,...,ไตรมาส 4,ไตร 4,4,2026,2026,26,2569,69,2025-12-22 10:06:39,2025-12-22 10:06:39
47,202612,1524,Dec,December,ธ.ค.,ธันวาคม,12,2026-12-01,2026-12-31,20264,...,ไตรมาส 4,ไตร 4,4,2026,2026,26,2569,69,2025-12-22 10:06:39,2025-12-22 10:06:39
